Objective: get a better understanding of the i94 dataset columns. In particular, for categorical columns, do simple group-by analysis do get a feel of what categorical values we have.

In [1]:
import os
import pyspark
import configparser
import pandas as pd
import pyspark.sql.functions as F
import pyspark.sql.types as T

from pyspark.sql import SparkSession

In [2]:
# Ensure Jupyter Notebook display pandas dataframe fully. Show all columns. Do not truncate column value.
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 1000)

# Dev config from the non-secret configuration file
config_dev = configparser.ConfigParser()
config_dev.read_file(open('aws_dev.cfg'))

PAR_I94_FILE_BY_NONE = config_dev.get('DATA_PATHS_LOCAL', 'PAR_I94_FILE_BY_NONE')

In [3]:
spark = SparkSession.builder.\
    config("spark.jars.repositories", "https://repos.spark-packages.org/").\
    config("spark.jars.packages", "saurfang:spark-sas7bdat:2.0.0-s_2.11").\
    enableHiveSupport().getOrCreate()

In [4]:
df_i94 = spark.read.parquet(PAR_I94_FILE_BY_NONE)

In [5]:
df_i94.printSchema()

root
 |-- cicid: double (nullable = true)
 |-- i94yr: double (nullable = true)
 |-- i94mon: double (nullable = true)
 |-- i94cit: double (nullable = true)
 |-- i94res: double (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: double (nullable = true)
 |-- i94mode: double (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: double (nullable = true)
 |-- i94bir: double (nullable = true)
 |-- i94visa: double (nullable = true)
 |-- count: double (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: double (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admnum: double (nullable = 

In [6]:
# Register the spark dataframe as a spark SQL data for ease of adhoc querying.
df_i94.createOrReplaceTempView("df_i94")

In [7]:
def simple_groupby(spark, df, colname: str, ordered: bool = True):
    """ Given a Spark Session, Spark DataFrame and a group-by field, 
        do a simple groupby
        Return a Spark DataFrame
    """
    df.createOrReplaceTempView("_tmp_df")
    sql_str = f"""\
        select {colname}, count(*) as records
        from _tmp_df
        group by 1
    """
    df2 = spark.sql(sql_str)
    if ordered:
        df2=df2.orderBy(colname)       
    return df2

# cicid

CIC-ID. Unique ID of each immigrant. It is unique per Year (i94yr) and Month (i94mon).

Observation from the groupby:

* looks like cicid is incremental in numeric value (integer).
* total records per cicid could be between 1 and 12 (validated by the spark_sql queries for `df_cic_agg_[1-5]`). min/max analysis confirms this hypothesis also.
* is cicid constant? ie. does the same cicid in January 2016 the same entity as the same one in March 2016, for example? (not sure). If so, the 12 i94 monthly datasets, when analysis altogether, appear to give a time-series view of a CICID.

In [8]:
df_cicid_agg_1 = simple_groupby(spark, df_i94, 'cicid')
df_cicid_agg_1.show(1000)

+------+-------+
| cicid|records|
+------+-------+
|   1.0|      1|
|   2.0|      4|
|   3.0|      3|
|   4.0|      3|
|   5.0|      5|
|   6.0|      6|
|   7.0|      7|
|   8.0|      5|
|   9.0|      4|
|  10.0|      2|
|  11.0|      3|
|  12.0|      3|
|  13.0|      3|
|  14.0|      2|
|  15.0|      5|
|  16.0|      5|
|  17.0|      5|
|  18.0|      6|
|  19.0|      6|
|  20.0|      7|
|  21.0|      5|
|  22.0|      7|
|  23.0|      5|
|  24.0|      6|
|  25.0|      4|
|  26.0|      4|
|  27.0|      3|
|  28.0|      5|
|  29.0|      6|
|  30.0|      4|
|  31.0|      3|
|  32.0|      3|
|  33.0|      5|
|  34.0|      3|
|  35.0|      3|
|  36.0|      4|
|  37.0|      5|
|  38.0|      5|
|  39.0|      5|
|  40.0|      6|
|  41.0|      5|
|  42.0|      6|
|  43.0|      5|
|  44.0|      6|
|  45.0|      4|
|  46.0|      6|
|  47.0|      6|
|  48.0|      6|
|  49.0|      4|
|  50.0|      3|
|  51.0|      4|
|  52.0|      7|
|  53.0|      9|
|  54.0|      7|
|  55.0|      6|
|  56.0|      

## CICID hypothesis

* My hypothesis is that cicid is unique per month.
* if my hypothesis is correct then the following query should requrn 0 rows

In [9]:
df_i94.createOrReplaceTempView("df_i94")

# My hypothesis is that cicid is unique per month.
# if my hypothesis is correct then the following query should requrn 0 rows
df_cicid_agg_2 = spark.sql("""\
select
    cicid,
    i94yr,
    i94mon,
    count(*) as records
from df_i94
group by 1, 2, 3
having records > 1
""")

df_cicid_agg_2.show()

+-----+-----+------+-------+
|cicid|i94yr|i94mon|records|
+-----+-----+------+-------+
+-----+-----+------+-------+



If my hypothesis is correct the following query may return some rows. (hypothesis validated)

In [10]:
# If my hypothesis is correct the following query may return some rows. (hypothesis validated)
df_cicid_agg_3 = spark.sql("""\
select
    cicid,
    count(*) as records
from df_i94
group by 1
having records > 1
""")

df_cicid_agg_3.show()

+---------+-------+
|    cicid|records|
+---------+-------+
|5680982.0|      7|
|5681018.0|      6|
|5681279.0|      6|
|5681289.0|      8|
|5681452.0|      5|
|5681523.0|      8|
|5681771.0|      6|
|5682204.0|      3|
|5682909.0|      7|
|5683017.0|      7|
|5683674.0|      6|
|5684532.0|      5|
|5684559.0|      3|
|5684829.0|      5|
|5685062.0|      6|
|5685608.0|      5|
|5686057.0|      7|
|5686102.0|      7|
|5686163.0|      7|
|5686168.0|      4|
+---------+-------+
only showing top 20 rows



Let's take a look at sample CICIDs with more than 1 records for the entire year.

In [11]:
df_cicid_agg_3.createOrReplaceTempView("df_cicid_agg_3")
df_cicid_agg_4 = spark.sql("""\
select
    b.cicid,
    b.i94yr,
    b.i94mon
from (select cicid from df_cicid_agg_3) a
inner join (select cicid, i94yr, i94mon from df_i94) b
    on a.cicid = b.cicid
order by 1, 2, 3
""")

df_cicid_agg_4.show()

+-----+------+------+
|cicid| i94yr|i94mon|
+-----+------+------+
|  2.0|2016.0|   2.0|
|  2.0|2016.0|   3.0|
|  2.0|2016.0|   5.0|
|  2.0|2016.0|   7.0|
|  3.0|2016.0|   3.0|
|  3.0|2016.0|   5.0|
|  3.0|2016.0|   7.0|
|  4.0|2016.0|   5.0|
|  4.0|2016.0|   6.0|
|  4.0|2016.0|   7.0|
|  5.0|2016.0|   2.0|
|  5.0|2016.0|   5.0|
|  5.0|2016.0|   6.0|
|  5.0|2016.0|   7.0|
|  5.0|2016.0|  10.0|
|  6.0|2016.0|   2.0|
|  6.0|2016.0|   4.0|
|  6.0|2016.0|   6.0|
|  6.0|2016.0|   7.0|
|  6.0|2016.0|  10.0|
+-----+------+------+
only showing top 20 rows



In [12]:
# what's the min and max number of records per CICID (during 2016? expecting 1-12)
df_cicid_agg_5 = spark.sql("""\
select
    min(records) as min_records,
    max(records) as max_records
from (
    select
        cicid,
        count(*) as records
    from df_i94
    group by 1
) a
""")

df_cicid_agg_5.show()

+-----------+-----------+
|min_records|max_records|
+-----------+-----------+
|          1|         12|
+-----------+-----------+



Mini conclusion: CICID is unique per monthly dataset. The 12 i94 monthly datasets, when analysis altogether, appear to give a time-series view of a CICID.

# i94yr

i94yr - 4 digit year

In [13]:
df_i94yr_agg_1 = simple_groupby(spark, df_i94, 'i94yr')
df_i94yr_agg_1.show()

+------+--------+
| i94yr| records|
+------+--------+
|2016.0|40790529|
+------+--------+



# i94mon

i94mon - Numeric month

In [14]:
df_i94mon_agg_1 = simple_groupby(spark, df_i94, 'i94mon')
df_i94mon_agg_1.show()

+------+-------+
|i94mon|records|
+------+-------+
|   1.0|2847924|
|   2.0|2570543|
|   3.0|3157072|
|   4.0|3096313|
|   5.0|3444249|
|   6.0|3574989|
|   7.0|4265031|
|   8.0|4103570|
|   9.0|3733786|
|  10.0|3649136|
|  11.0|2914926|
|  12.0|3432990|
+------+-------+



# i94cit and i94res

i94cit & i94res - This format shows all the valid and invalid (country?) codes for processing

Idea: parse lookup table (PROC FORMAT) from `I94_SAS_Labels_Descriptions.SAS` into a structured dataset (e.g. tabular dataframe / csv) for downstream processing.

Rationale for idea: we may potentially map `i94cit` and `i94res` (numeric code) to some meaningful string values (country?)

Observation:

* Key: integer code (1-3 digits)
* Value: country name, or state name.

In [15]:
df_i94cit_agg_1 = simple_groupby(spark, df_i94, 'i94cit')
df_i94cit_agg_1.show(1000)

+------+-------+
|i94cit|records|
+------+-------+
|  null|  28575|
|   0.0|   1288|
| 101.0|  11424|
| 102.0|   1287|
| 103.0| 203883|
| 104.0| 270440|
| 105.0|  30101|
| 106.0|      5|
| 107.0| 190060|
| 108.0| 328086|
| 109.0|  23583|
| 110.0| 154249|
| 111.0|1679312|
| 112.0|  10537|
| 113.0|  82025|
| 114.0|  85107|
| 115.0|  58268|
| 116.0| 476312|
| 117.0|1116790|
| 118.0|  20870|
| 119.0|   1832|
| 120.0|   3394|
| 121.0|  13586|
| 122.0|   1086|
| 123.0| 697890|
| 124.0| 272444|
| 125.0|    407|
| 126.0| 163094|
| 127.0|  74216|
| 128.0|    642|
| 129.0| 895509|
| 130.0| 561519|
| 131.0| 410328|
| 133.0| 205481|
| 134.0|      1|
| 135.0|4531534|
| 139.0|     23|
| 140.0| 101511|
| 141.0|  47497|
| 143.0|     16|
| 145.0|   6230|
| 147.0|     18|
| 148.0|2051390|
| 149.0|     15|
| 151.0|  10034|
| 152.0|   7127|
| 153.0|  17849|
| 154.0|   7083|
| 155.0|  21535|
| 156.0|   2709|
| 157.0|   8063|
| 158.0|   2700|
| 159.0|   1580|
| 162.0|  69741|
| 163.0|   9626|
| 164.0|   788

In [16]:
df_i94res_agg_1 = simple_groupby(spark, df_i94, 'i94res')
df_i94res_agg_1.show(1000)

+------+-------+
|i94res|records|
+------+-------+
| 101.0|  12317|
| 102.0|   1589|
| 103.0| 199332|
| 104.0| 276722|
| 105.0|  28080|
| 107.0| 180847|
| 108.0| 327074|
| 109.0|  22441|
| 110.0| 150704|
| 111.0|1641152|
| 112.0|2046288|
| 113.0|  74444|
| 114.0|  78625|
| 115.0|  57856|
| 116.0| 443005|
| 117.0| 988860|
| 118.0|  19420|
| 119.0|   2082|
| 120.0|  27660|
| 121.0|  19360|
| 122.0|   3652|
| 123.0| 674544|
| 124.0| 278000|
| 126.0| 128481|
| 127.0|  69160|
| 128.0|    644|
| 129.0| 807447|
| 130.0| 560581|
| 131.0| 471070|
| 135.0|4587092|
| 139.0|     44|
| 140.0| 100034|
| 141.0|  45698|
| 143.0|   1298|
| 145.0|   6250|
| 149.0|    101|
| 151.0|   9873|
| 152.0|   7093|
| 153.0|  17357|
| 154.0|   7956|
| 155.0|  21161|
| 156.0|   2896|
| 157.0|   9291|
| 158.0| 273518|
| 159.0|   1594|
| 161.0|   1199|
| 162.0|  87440|
| 163.0|   9484|
| 164.0|   7262|
| 165.0|  22312|
| 166.0|  20631|
| 167.0|   6599|
| 201.0|   4579|
| 203.0|   1797|
| 204.0| 101003|
| 206.0| 13265

# i94port

i94port - This format shows all the valid and invalid (Shipping Port?) codes for processing

Idea: parse lookup table (PROC FORMAT) from `I94_SAS_Labels_Descriptions.SAS` into a structured dataset (e.g. tabular dataframe / csv) for downstream processing.

Rationale for idea: we may potentially map `i94port` (3 character code) to some meaningful string values (port name?)

In [17]:
df_i94port_agg_1 = simple_groupby(spark, df_i94, 'i94port')
df_i94port_agg_1.show(1000)

+-------+-------+
|i94port|records|
+-------+-------+
|    48Y|      2|
|    5KE|     58|
|    5T6|    161|
|    74S|      1|
|    ABE|      2|
|    ABG|    527|
|    ABQ|     61|
|    ABS|    264|
|    ACY|      5|
|    ADS|    388|
|    ADT|    124|
|    ADW|    505|
|    AFW|     19|
|    AGA|1337940|
|    AGM|      2|
|    AGN|    163|
|    AGU|     12|
|    ALB|     49|
|    ALC|   3976|
|    ANA|      2|
|    ANC|  23388|
|    AND|    912|
|    ANP|      6|
|    ANT|     25|
|    ANZ|  10650|
|    APA|      2|
|    APF|    133|
|    ARB|      4|
|    ASI|      1|
|    ATL|1141425|
|    ATW|     30|
|    AUH|     92|
|    AUS|  33628|
|    AXB|   5343|
|    BAL|  44363|
|    BAU|     58|
|    BDL|    189|
|    BEB|    168|
|    BED|    480|
|    BEE|    163|
|    BEL|     15|
|    BGM|  12109|
|    BHX|     44|
|    BKF|      1|
|    BLA| 225675|
|    BLI|     45|
|    BOA|   5040|
|    BOC|      1|
|    BOS| 952631|
|    BQN|    556|
|    BRG|    111|
|    BRO|   8306|
|    BTN| 

# arrdate

arrdate is the Arrival Date in the USA. It is a SAS date numeric field that a permament format has not been applied.  Please apply whichever date format works for you.

In [18]:
df_arrdate_agg_1 = simple_groupby(spark, df_i94, 'arrdate')
df_arrdate_agg_1.show(1000)

+-------+-------+
|arrdate|records|
+-------+-------+
|20454.0|  83984|
|20455.0|  75646|
|20456.0|  79959|
|20457.0|  89325|
|20458.0|  85650|
|20459.0|  84420|
|20460.0|  93107|
|20461.0|  93818|
|20462.0| 101576|
|20463.0| 106167|
|20464.0|  99145|
|20465.0|  84688|
|20466.0|  85486|
|20467.0|  92151|
|20468.0|  95030|
|20469.0| 107911|
|20470.0| 110195|
|20471.0| 108063|
|20472.0|  85322|
|20473.0|  90017|
|20474.0|  93107|
|20475.0|  93779|
|20476.0|  83658|
|20477.0|  97471|
|20478.0| 100611|
|20479.0|  79057|
|20480.0|  79898|
|20481.0|  86300|
|20482.0|  88187|
|20483.0| 100130|
|20484.0|  94066|
|20485.0|  91961|
|20486.0|  76452|
|20487.0|  84123|
|20488.0|  99169|
|20489.0| 107072|
|20490.0| 114292|
|20491.0|  98497|
|20492.0|  91880|
|20493.0|  75008|
|20494.0|  84583|
|20495.0|  91642|
|20496.0|  99711|
|20497.0| 105536|
|20498.0|  92379|
|20499.0|  92812|
|20500.0|  72838|
|20501.0|  78243|
|20502.0|  83633|
|20503.0|  87697|
|20504.0|  94507|
|20505.0|  90378|
|20506.0| 

# i94mode

i94mode - There are missing values as well as not reported.

Idea: parse lookup table (PROC FORMAT) from `I94_SAS_Labels_Descriptions.SAS` into a structured dataset (e.g. tabular dataframe / csv) for downstream processing.

Rationale for idea: we may potentially map `i94mode` (numeric code) to some meaningful string values (mode of transport)

* Key: integer
* Value: mode of transport:

```
    1 = 'Air'
    2 = 'Sea'
    3 = 'Land'
    9 = 'Not reported'
```

In [19]:
df_i94mode_agg_1 = simple_groupby(spark, df_i94, 'i94mode')
df_i94mode_agg_1.show(1000)

+-------+--------+
|i94mode| records|
+-------+--------+
|   null|   73949|
|    0.0|       4|
|    1.0|39166088|
|    2.0|  387184|
|    3.0| 1095001|
|    9.0|   68303|
+-------+--------+



# i94addr

i94addr - There are lots of invalid codes in this variable and the list below shows what we have found to be valid, everything else goes into 'other'

Idea: parse lookup table (PROC FORMAT) from `I94_SAS_Labels_Descriptions.SAS` into a structured dataset (e.g. tabular dataframe / csv) for downstream processing.

Rationale for idea: we may potentially map `i94addr` (1-2 character code) to some meaningful string values (name of the US State or oversea territories)


In [20]:
df_i94addr_agg_1 = simple_groupby(spark, df_i94, 'i94addr')
df_i94addr_agg_1.show(1000)

+-------+-------+
|i94addr|records|
+-------+-------+
|   null|2027926|
|     **|      4|
|     ..|    254|
|     .7|      1|
|     .9|      3|
|     .A|      5|
|     .C|    206|
|     .D|     14|
|     .F|      4|
|     .H|      7|
|     .I|     27|
|     .K|      2|
|     .L|      4|
|     .M|     11|
|     .N|     36|
|     .O|      3|
|     .S|      7|
|     .T|     56|
|     .V|      5|
|     .W|      1|
|      /|      1|
|      0|    230|
|     00|     30|
|     01|      1|
|     02|     11|
|     03|      6|
|     04|      4|
|     06|      8|
|     07|     16|
|     08|      9|
|      1|      8|
|     10|     89|
|     11|     56|
|     12|     13|
|     13|      5|
|     14|      8|
|     15|      1|
|     16|      2|
|     17|      6|
|     18|     14|
|     19|      9|
|      2|     20|
|     20|     33|
|     21|      9|
|     22|      7|
|     23|      9|
|     26|      3|
|     27|      9|
|     28|      4|
|     29|      5|
|      3|      3|
|     30|     11|
|     31| 

# depdate

DEPDATE is the Departure Date from the USA. It is a SAS date numeric field that a permament format has not been applied.  Please apply whichever date format works for you.

idea: convert DEPDATE into Python DateTime. (See: https://knowledge.udacity.com/questions/66798)

Observation: lots of depdate with `null` value.

In [21]:
df_depdate_agg_1 = simple_groupby(spark, df_i94, 'depdate')
df_depdate_agg_1.show(1000)

+--------+-------+
| depdate|records|
+--------+-------+
|    null|3308012|
|-14388.0|      1|
|-14375.0|      1|
|-14359.0|      1|
|-14342.0|      4|
|-14334.0|      3|
|-14328.0|      1|
|-14321.0|      1|
|-14312.0|      3|
|-14307.0|     28|
|-14283.0|      1|
|-14021.0|      1|
|-14011.0|      1|
|-13977.0|      1|
|-13888.0|      1|
|-12285.0|      1|
|-11972.0|      1|
|-11950.0|      1|
| -6717.0|      1|
|   598.0|      1|
|   669.0|      1|
|   962.0|      1|
|  4260.0|      1|
|  4264.0|      1|
|  4331.0|      1|
|  7924.0|      1|
| 14979.0|      1|
| 14991.0|      1|
| 15065.0|      1|
| 15176.0|      1|
| 15279.0|      1|
| 16809.0|      1|
| 17032.0|      1|
| 17076.0|      1|
| 17119.0|      1|
| 17122.0|      1|
| 17143.0|      1|
| 17335.0|      1|
| 18289.0|      1|
| 18318.0|      1|
| 18370.0|      1|
| 18415.0|      1|
| 18535.0|      1|
| 18537.0|      1|
| 18545.0|      1|
| 18594.0|      1|
| 18634.0|      1|
| 18688.0|      2|
| 18689.0|      1|
| 18804.0|  

# i94bir

I94BIR - Age of Respondent in Years

Observation: there are some invalue values (negative 1/2/3 years old?). null should be ok as these are just unprovided data points?

In [22]:
df_i94bir_agg_1 = simple_groupby(spark, df_i94, 'i94bir')
df_i94bir_agg_1.show(1000)

+------+-------+
|i94bir|records|
+------+-------+
|  null|   9517|
|  -3.0|      1|
|  -2.0|      2|
|  -1.0|      7|
|   0.0|  44698|
|   1.0| 173201|
|   2.0| 178813|
|   3.0| 168773|
|   4.0| 195652|
|   5.0| 210899|
|   6.0| 227145|
|   7.0| 235070|
|   8.0| 252727|
|   9.0| 268906|
|  10.0| 276404|
|  11.0| 282934|
|  12.0| 304090|
|  13.0| 313089|
|  14.0| 331748|
|  15.0| 386115|
|  16.0| 431350|
|  17.0| 414014|
|  18.0| 406308|
|  19.0| 444784|
|  20.0| 491466|
|  21.0| 552656|
|  22.0| 598450|
|  23.0| 653539|
|  24.0| 667398|
|  25.0| 707037|
|  26.0| 772811|
|  27.0| 813885|
|  28.0| 856094|
|  29.0| 857364|
|  30.0| 889353|
|  31.0| 851608|
|  32.0| 842397|
|  33.0| 843090|
|  34.0| 845296|
|  35.0| 823562|
|  36.0| 804061|
|  37.0| 791501|
|  38.0| 768890|
|  39.0| 762636|
|  40.0| 800307|
|  41.0| 761353|
|  42.0| 769873|
|  43.0| 771510|
|  44.0| 779844|
|  45.0| 788721|
|  46.0| 765836|
|  47.0| 748139|
|  48.0| 745679|
|  49.0| 717835|
|  50.0| 749601|
|  51.0| 71638

# i94visa

I94VISA - Visa codes collapsed into three categories:

```
   1 = Business
   2 = Pleasure
   3 = Student
```

In [23]:
df_i94visa_agg_1 = simple_groupby(spark, df_i94, 'i94visa')
df_i94visa_agg_1.show(1000)

+-------+--------+
|i94visa| records|
+-------+--------+
|    1.0| 5575279|
|    2.0|33641979|
|    3.0| 1573271|
+-------+--------+



# count

COUNT - Used for summary statistics.

Obervation: looks like this column is always `1`. This is a classic technique for easie of aggregation (so we summing this column is equivalent as counting rows).

In [24]:
df_count_agg_1 = simple_groupby(spark, df_i94, 'count')
df_count_agg_1.show(1000)

+-----+--------+
|count| records|
+-----+--------+
|  1.0|40790529|
+-----+--------+



# dtadfile

DTADFILE - Character Date Field - Date added to I-94 Files - CIC does not use

Observation: looks like numbers are in date like format: yyyymmdd

In [25]:
df_dtadfile_agg_1 = simple_groupby(spark, df_i94, 'dtadfile')
df_dtadfile_agg_1.show(1000)

+--------+-------+
|dtadfile|records|
+--------+-------+
|    null| 131050|
|20081124|      1|
|20120102|      1|
|20120418|      1|
|20120831|      1|
|20130312|      1|
|20130514|      1|
|20130811|      1|
|20140407|      1|
|20150107|      1|
|20150406|      1|
|20150530|      1|
|20150608|      1|
|20150619|      1|
|20150806|      1|
|20150810|      1|
|20150822|      1|
|20150824|      1|
|20150912|      1|
|20151015|      3|
|20160101|  76374|
|20160102|  67805|
|20160103|  71405|
|20160104|  80177|
|20160105|  78033|
|20160106|  76649|
|20160107|  84591|
|20160108|  85774|
|20160109|  93539|
|20160110|  96386|
|20160111|  91753|
|20160112|  77339|
|20160113|  78654|
|20160114|  84839|
|20160115|  79800|
|20160116|  61360|
|20160117|  59553|
|20160118|  55163|
|20160119|  81077|
|20160120| 179619|
|20160121|  85785|
|20160122|  86732|
|20160123|  76574|
|20160124|  88100|
|20160125|  92923|
|20160126|  72474|
|20160127|  73267|
|20160128|  79315|
|20160129|  82069|
|20160130|  

# visapost

VISAPOST - Department of State where where Visa was issued - CIC does not use

In [26]:
df_visapost_agg_1 = simple_groupby(spark, df_i94, 'visapost')
df_visapost_agg_1.show(1000)

+--------+--------+
|visapost| records|
+--------+--------+
|    null|24032175|
|     999|    8823|
|     ABD|   45618|
|     ABG|     598|
|     ABJ|    6025|
|     ABS|     121|
|     ABU|   43207|
|     ACC|   16622|
|     ACK|   16054|
|     ADA|      31|
|     ADD|   16650|
|     ADL|       6|
|     ADN|       3|
|     ADT|       3|
|     AGA|    1166|
|     AKD|    1099|
|     ALB|      19|
|     ALC|      10|
|     ALG|    7444|
|     ALP|       4|
|     ALX|       7|
|     AMB|      41|
|     AMM|   24310|
|     AMS|   23333|
|     ANC|      38|
|     AND|       9|
|     ANK|   41587|
|     ANT|     739|
|     AON|       2|
|     APA|     197|
|     APC|       1|
|     ARE|      56|
|     ASC|      13|
|     ASM|    1773|
|     ASN|   30455|
|     AST|    6394|
|     ATA|    9770|
|     ATF|       5|
|     ATH|   15591|
|     ATL|     635|
|     ATW|       2|
|     AUC|     179|
|     BAL|     309|
|     BAM|    2386|
|     BAN|    1316|
|     BAR|       2|
|     BAT|      17|


# occup

OCCUP - Occupation that will be performed in U.S. - CIC does not use

In [27]:
# WARNING: can take very long!
df_occup_agg_1 = simple_groupby(spark, df_i94, 'occup')
df_occup_agg_1.show(1000)

+-----+--------+
|occup| records|
+-----+--------+
| null|40597574|
|  010|       1|
|  035|       1|
|  037|       1|
|  039|       2|
|  049|       6|
|  093|       1|
|  100|       1|
|  101|       1|
|  111|       1|
|  120|       1|
|  130|       1|
|  200|       2|
|  300|       3|
|  430|       5|
|  800|       1|
|  850|       3|
|  855|     260|
|  992|      80|
|  995|       1|
|  997|       2|
|  999|     168|
|  ACH|     325|
|  ACN|      82|
|  ADM|    1207|
|  AGR|     147|
|  ANC|      16|
|  ARC|       2|
|  ART|       4|
|  ASP|     194|
|  AST|      16|
|  ATH|     147|
|  ATR|       8|
|  BIL|      84|
|  BUG|       5|
|  CAN|      19|
|  CAT|       2|
|  CEO|     611|
|  CHE|      37|
|  CLN|      17|
|  CMP|     510|
|  CNS|      95|
|  CPA|     361|
|  DDS|     167|
|  DOC|       1|
|  DOD|       6|
|  DVM|      28|
|  EAG|      61|
|  EAS|      55|
|  ECH|      93|
|  ECO|       7|
|  ECV|     161|
|  EDR|      27|
|  EIN|     211|
|  ELE|     192|
|  ELT|     18

# entdepa

ENTDEPA - Arrival Flag - admitted or paroled into the U.S. - CIC does not use.

In [28]:
# WARNING: can take very long!
df_entdepa_agg_1 = simple_groupby(spark, df_i94, 'entdepa')
df_entdepa_agg_1.show(1000)

+-------+--------+
|entdepa| records|
+-------+--------+
|   null|    2404|
|      A| 1441544|
|      B|    1957|
|      F|      26|
|      G|30937479|
|      H|  190602|
|      I|   52706|
|      J|     118|
|      K|  265447|
|      M|     251|
|      N|    3649|
|      O| 5692584|
|      P|  197710|
|      Q|     274|
|      R|     316|
|      T|  896198|
|      U|   38896|
|      Z| 1068368|
+-------+--------+



# entdepd

ENTDEPA - Departure Flag - Departed, lost I-94 or is deceased - CIC does not use

In [29]:
df_entdepd_agg_1 = simple_groupby(spark, df_i94, 'entdepd')
df_entdepd_agg_1.show(1000)

+-------+--------+
|entdepd| records|
+-------+--------+
|   null| 3287909|
|      D| 1226934|
|      G|   15089|
|      I| 1395706|
|      J|   21315|
|      K| 1053478|
|      L|     304|
|      M|     223|
|      N|  654917|
|      O|31560000|
|      Q|  896570|
|      R|  551183|
|      T|     656|
|      V|   39257|
|      W|   86873|
|      Y|       2|
|      Z|     113|
+-------+--------+



# entdepu

ENTDEPU - Update Flag - Either apprehended, overstayed, adjusted to perm residence - CIC does not use

In [30]:
# WARNING: can take very long?
df_entdepu_agg_1 = simple_groupby(spark, df_i94, 'entdepu')
df_entdepu_agg_1.show(1000)

+-------+--------+
|entdepu| records|
+-------+--------+
|   null|40777323|
|      P|       2|
|      U|   13156|
|      Y|      48|
+-------+--------+



# matflag

MATFLAG - Match flag - Match of arrival and departure records

In [31]:
df_matflag_agg_1 = simple_groupby(spark, df_i94, 'matflag')
df_matflag_agg_1.show(1000)

+-------+--------+
|matflag| records|
+-------+--------+
|   null| 3219581|
|      M|37570948|
+-------+--------+



# biryear

BIRYEAR - 4 digit year of birth. 

Observation:

* 2 invalid bird year: 204?

```
+-------+-------+
|biryear|records|
+-------+-------+
|  204.0|      2|
```

In [32]:
df_biryear_agg_1 = simple_groupby(spark, df_i94, 'biryear')
df_biryear_agg_1.show(1000)

+-------+-------+
|biryear|records|
+-------+-------+
|   null|   9517|
|  204.0|      2|
| 1900.0|      9|
| 1901.0|      5|
| 1902.0|      4|
| 1903.0|      8|
| 1904.0|     10|
| 1905.0|     10|
| 1906.0|     11|
| 1907.0|     15|
| 1908.0|     16|
| 1909.0|     12|
| 1910.0|     19|
| 1911.0|    108|
| 1912.0|     92|
| 1913.0|     65|
| 1914.0|     77|
| 1915.0|     86|
| 1916.0|    185|
| 1917.0|    186|
| 1918.0|    253|
| 1919.0|    605|
| 1920.0|    750|
| 1921.0|    936|
| 1922.0|   1386|
| 1923.0|   1885|
| 1924.0|   2786|
| 1925.0|   3983|
| 1926.0|   5500|
| 1927.0|   7310|
| 1928.0|  10300|
| 1929.0|  13513|
| 1930.0|  18165|
| 1931.0|  21947|
| 1932.0|  28147|
| 1933.0|  34047|
| 1934.0|  43170|
| 1935.0|  53658|
| 1936.0|  66369|
| 1937.0|  76462|
| 1938.0|  90638|
| 1939.0| 105916|
| 1940.0| 123528|
| 1941.0| 137578|
| 1942.0| 160812|
| 1943.0| 181918|
| 1944.0| 208579|
| 1945.0| 222752|
| 1946.0| 278391|
| 1947.0| 314862|
| 1948.0| 332536|
| 1949.0| 358235|
| 1950.0| 

# dtaddto

DTADDTO - Character Date Field - Date to which admitted to U.S. (allowed to stay until) - CIC does not use

Observations:

* in general numeric value appear to be in US dateformat: mmddyyyy
* null values probably meant not applicable?
* Contains invalid values

```
+--------+-------+
| dtaddto|records|
+--------+-------+
|-00-0000|      3|
|/   183D|      1|
|   /184D|      1|
```

In [33]:
df_dtaddto_agg_1 = simple_groupby(spark, df_i94, 'dtaddto')
df_dtaddto_agg_1.show(1000)

+--------+-------+
| dtaddto|records|
+--------+-------+
|    null| 101551|
|-00-0000|      3|
|/   183D|      1|
|   /184D|      1|
|00000000|  40209|
|01011970|      2|
|01012016|     16|
|01012017| 127109|
|01012018|   2977|
|01012019|      3|
|01012020|      2|
|01012099|      1|
|01022016|     11|
|01022017| 121018|
|01022018|   3938|
|01022019|      4|
|01032016|     20|
|01032017| 114356|
|01032018|   2998|
|01032019|      1|
|01032021|      3|
|01042016|     23|
|01042017| 124336|
|01042018|   2035|
|01042019|      1|
|01052016|     25|
|01052017| 139699|
|01052018|   1558|
|01052019|      1|
|01052021|      1|
|01062016|     23|
|01062017| 137528|
|01062018|   1165|
|01062019|      8|
|01072016|     52|
|01072017| 139680|
|01072018|   1022|
|01082016|     49|
|01082017| 120038|
|01082018|   1217|
|01082019|      1|
|01082185|      1|
|01092016|     57|
|01092017| 121911|
|01092018|   1252|
|01092019|      1|
|01102016|     93|
|01102017| 126045|
|01102018|   1083|
|01102019|  

# gender

GENDER - Non-immigrant sex

Question: what are "U" and "X"?

```
+------+--------+
|gender| Meaning|
+------+--------+
|     F|Female|
|     M|Male|
|     U|???|
|     X|???|
+------+--------+
```

In [34]:
df_gender_agg_1 = simple_groupby(spark, df_i94, 'gender')
df_gender_agg_1.show(1000)

+------+--------+
|gender| records|
+------+--------+
|  null| 4079983|
|     F|18176125|
|     M|18504505|
|     U|   18906|
|     X|   11010|
+------+--------+



# insnum

INSNUM - INS number

Observations:

* appear numeric in general with zero padded on left.
* abnormal values:

```
+------+--------+
|insnum| records|
+------+--------+
|     '|       5|
| '2868|       1|
| '3884|       1|
| .3434|       1|
| .3890|       1|
| /3685|       1|
|     0|       6|
```

In [35]:
df_insnum_agg_1 = simple_groupby(spark, df_i94, 'insnum')
df_insnum_agg_1.show(1000)

+------+--------+
|insnum| records|
+------+--------+
|  null|35678095|
|     '|       5|
| '2868|       1|
| '3884|       1|
| .3434|       1|
| .3890|       1|
| /3685|       1|
|     0|       6|
|000078|    2784|
| 00037|     132|
|  0004|      53|
| 00041|      35|
| 00058|      11|
|  0007|       4|
| 00076|      11|
| 00081|     140|
| 00083|       2|
|000861|       1|
|  0009|       7|
| 00091|    2742|
|000923|       2|
| 00093|     102|
| 00110|       1|
| 00111|     156|
|  0012|       1|
| 00129|     107|
| 00136|    1238|
| 00138|     681|
| 00146|       2|
|  0015|       7|
| 00150|     380|
| 00159|       1|
| 00165|       7|
| 00167|       6|
|  0019|      25|
| 00192|     547|
| 00194|      29|
| 00195|       1|
|001991|       1|
| 00203|       1|
| 00209|       1|
|  0022|       5|
| 00220|      42|
| 00221|     969|
| 00231|      17|
| 00233|       5|
| 00236|    3077|
| 00239|     210|
|  0024|       9|
| 00240|     478|
| 00249|    1033|
| 00251|       2|
| 00255|  

# airline

AIRLINE - Airline used to arrive in U.S.

In [36]:
df_airline_agg_1 = simple_groupby(spark, df_i94, 'airline')
df_airline_agg_1.show(1000)

+-------+-------+
|airline|records|
+-------+-------+
|   null|1308066|
|    *FF|    952|
|    *GA| 102156|
|    *UU|    342|
|    001|      6|
|    006|     26|
|    009|    101|
|    00F|      1|
|    00I|      2|
|    00K|      1|
|    00X|      8|
|    012|      1|
|    017|      8|
|    018|     26|
|    01B|      1|
|    020|      1|
|    026|    152|
|    027|      2|
|    029|     31|
|    02Z|      2|
|    031|     11|
|    039|      3|
|    04K|      3|
|     05|      5|
|    050|      1|
|    05I|      1|
|    05J|      2|
|    06S|     16|
|    072|      3|
|    078|      2|
|    0AF|     30|
|    0AX|      6|
|    0AZ|     30|
|    0B0|      1|
|    0B8|    248|
|    0BA|    190|
|    0BD|      1|
|    0CC|      1|
|    0CL|     11|
|    0DF|      4|
|    0DP|     16|
|    0FF|     28|
|    0FR|    105|
|    0FT|     15|
|    0FV|     10|
|    0FY|    530|
|    0GJ|      6|
|    0HC|    172|
|     0J|     20|
|    0L7|     38|
|    0MT|      2|
|    0MY|      1|
|    0N3| 

# admnum

ADMNUM - Admission Number

In [37]:
df_admnum_agg_1 = simple_groupby(spark, df_i94, 'admnum')
df_admnum_agg_1.show(1000)

+------------+-------+
|      admnum|records|
+------------+-------+
|         0.0|    498|
|        27.0|      1|
|        33.0|      1|
|     79733.0|      1|
|    197185.0|      1|
|    452033.0|      1|
|    971885.0|      1|
|   1004233.0|      1|
|   1078085.0|      1|
|   1218224.0|      1|
|   1219024.0|      1|
|   1219124.0|      1|
|   1219224.0|      1|
|   1219324.0|      1|
|   1219424.0|      1|
|   1222424.0|      1|
|   1226124.0|      1|
|   1226224.0|      1|
|   1233085.0|      1|
|   1236624.0|      1|
|   1236724.0|      1|
|   1236824.0|      1|
|   1236924.0|      1|
|   1237124.0|      1|
|   1246424.0|      1|
|   1248124.0|      1|
|   1248324.0|      1|
|   1248824.0|      1|
|   1248924.0|      1|
|   1292524.0|      1|
|   1292624.0|      1|
|   1293124.0|      1|
|   1293224.0|      1|
|   1295524.0|      1|
|   1300424.0|      1|
|   1300624.0|      1|
|   1300824.0|      1|
|   1302524.0|      1|
|   1305424.0|      1|
|   1305524.0|      1|
|   1306724

# fltno

FLTNO - Flight number of Airline used to arrive in U.S.



In [38]:
df_fltno_agg_1 = simple_groupby(spark, df_i94, 'fltno')
df_fltno_agg_1.show(1000)

+-----+-------+
|fltno|records|
+-----+-------+
| null| 333922|
|    -|      1|
|-PF10|      3|
|-XCHI|      2|
|-XCNP|      6|
|-XCPF|     41|
|0 111|      2|
|0 178|      1|
|0 188|      2|
|0 294|      1|
|0 301|      2|
|0 314|      1|
|0 336|      1|
|0 608|      1|
|0 612|      1|
|0 622|      1|
|0 874|      2|
|0 941|      1|
|   00|      2|
|00 20|      1|
|00 71|      1|
|  000|      6|
| 0000|      2|
|00000|     38|
|00001| 277181|
|00002| 171642|
|00003| 178128|
|00004|  79819|
|00005| 158119|
|00006| 276803|
|00007| 192065|
|00008| 192210|
|00009|  76193|
|0000L|      2|
|00010| 122486|
|00011| 176260|
|00012| 134472|
|00013|    123|
|00014|  26420|
|00015| 150617|
|00016|  85417|
|00017|  80471|
|00018| 127392|
|00019|  82542|
|0001C|    741|
|00020|  42517|
|00021|  62741|
|00022|  68629|
|00023|  85043|
|00024|   9594|
|00025|  46282|
|00026| 155277|
|00027|  96887|
|00028|  87837|
|00029|  20940|
|0002B|     77|
|0002C|    148|
|00030|  28291|
|00031|  58216|
|00032| 

# visatype

VISATYPE - Class of admission legally admitting the non-immigrant to temporarily stay in U.S.

In [39]:
df_visatype_agg_1 = simple_groupby(spark, df_i94, 'visatype')
df_visatype_agg_1.show(1000)

+--------+--------+
|visatype| records|
+--------+--------+
|      B1| 2282096|
|      B2|15188785|
|      CP|  272007|
|     CPL|     236|
|      E1|   48905|
|      E2|  259215|
|      F1| 1487432|
|      F2|   68866|
|     GMB|    2728|
|     GMT| 1265275|
|       I|   39054|
|      I1|    2825|
|      M1|   16306|
|      M2|     667|
|     SBP|      61|
|      WB| 2940456|
|      WT|16915615|
+--------+--------+



# Write groupby stats into CSV files

So that our analytical teams may quickly get a feel of the distinct values (and associated counts) for 

In [40]:
def write_groupby_stats_csv(
        df_spark,
        groupby_field_name,
        out_filename,
        n_rows=1000,
        out_dir='raw_input_data/i94_sample_groupby_stats_csv'
    ):
    """ given one spark dataframe (in a specific schema) save into a CSV file """
    
    print(f'writing: {out_filename}')
    pd.DataFrame(df_spark.take(n_rows), columns=[groupby_field_name, 'count'])\
        .to_csv(f"{out_dir}/{out_filename}", index=False)

    

def bulk_write_groupby_stats_csv():
    """ bulk run write_groupby_stats_csv """
    for e in [
        (df_cicid_agg_1, 'cicid', "df_cicid_agg_1.csv"),
        (df_i94yr_agg_1, 'i94yr', "df_i94yr_agg_1.csv"),
        (df_i94mon_agg_1, 'i94mon', "df_i94mon_agg_1.csv"),
        (df_i94cit_agg_1, 'i94cit', "df_i94cit_agg_1.csv"),
        (df_i94res_agg_1, 'i94res', "df_i94res_agg_1.csv"),        
        (df_i94port_agg_1, 'i94port', "df_i94port_agg_1.csv"),
        (df_arrdate_agg_1, 'arrdate', "df_arrdate_agg_1.csv"),
        (df_i94mode_agg_1, 'i94mode', "df_i94mode_agg_1.csv"),
        (df_i94addr_agg_1, 'i94addr', "df_i94addr_agg_1.csv"),
        (df_depdate_agg_1, 'depdate', "df_depdate_agg_1.csv"),
        (df_i94bir_agg_1, 'i94bir', "df_i94bir_agg_1.csv"),
        (df_i94visa_agg_1, 'i94visa', "df_i94visa_agg_1.csv"),
        (df_count_agg_1, 'count', "df_count_agg_1.csv"),
        (df_dtadfile_agg_1, 'dtadfile', "df_dtadfile_agg_1.csv"),
        (df_visapost_agg_1, 'visapost', "df_visapost_agg_1.csv"),
        (df_occup_agg_1, 'occup', "df_occup_agg_1.csv"),
        (df_entdepa_agg_1, 'entdepa', "df_entdepa_agg_1.csv"),
        (df_entdepd_agg_1, 'entdepd', "df_entdepd_agg_1.csv"),
        (df_entdepu_agg_1, 'entdepu', "df_entdepu_agg_1.csv"),
        (df_matflag_agg_1, 'matflag', "df_matflag_agg_1.csv"),        
        (df_biryear_agg_1, 'biryear', "df_biryear_agg_1.csv"),   
        (df_dtaddto_agg_1, 'dtaddto', "df_dtaddto_agg_1.csv"),   
        (df_gender_agg_1, 'gender', "df_gender_agg_1.csv"),   
        (df_insnum_agg_1, 'insnum', "df_insnum_agg_1.csv"),   
        (df_airline_agg_1, 'airline', "df_airline_agg_1.csv"),   
        (df_admnum_agg_1, 'admnum', "df_admnum_agg_1.csv"),   
        (df_fltno_agg_1, 'fltno', "df_fltno_agg_1.csv"),
        (df_visatype_agg_1, 'visatype', "df_visatype_agg_1.csv")
    ]:
        write_groupby_stats_csv(e[0], e[1], e[2])
        

In [41]:
bulk_write_groupby_stats_csv()

writing: df_cicid_agg_1.csv
writing: df_i94yr_agg_1.csv
writing: df_i94mon_agg_1.csv
writing: df_i94cit_agg_1.csv
writing: df_i94res_agg_1.csv
writing: df_i94port_agg_1.csv
writing: df_arrdate_agg_1.csv
writing: df_i94mode_agg_1.csv
writing: df_i94addr_agg_1.csv
writing: df_depdate_agg_1.csv
writing: df_i94bir_agg_1.csv
writing: df_i94visa_agg_1.csv
writing: df_count_agg_1.csv
writing: df_dtadfile_agg_1.csv
writing: df_visapost_agg_1.csv
writing: df_occup_agg_1.csv
writing: df_entdepa_agg_1.csv
writing: df_entdepd_agg_1.csv
writing: df_entdepu_agg_1.csv
writing: df_matflag_agg_1.csv
writing: df_biryear_agg_1.csv
writing: df_dtaddto_agg_1.csv
writing: df_gender_agg_1.csv
writing: df_insnum_agg_1.csv
writing: df_airline_agg_1.csv
writing: df_admnum_agg_1.csv
writing: df_fltno_agg_1.csv
writing: df_visatype_agg_1.csv


In [42]:
# gracefully terminate spark
spark.stop()